In [1]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [2]:
import cafle as cf
from cafle.genfunc import rounding as R

In [3]:
# import cf.management as mng
from cafle import management_actualcase1 as mng1

In [4]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), 
      pd.get_option("display.max_rows"))

40 200


# Introduction
* This is an example created by applying an actual case.

# Initial Setting

In [5]:
PY = 1/3.305785

In [6]:
class Assumption:
    def __init__(self):
        pass
    
    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
astn = Assumption()

In [7]:
class Assumption_sales:
    def __init__(self):
        self.prdtname = []
        self.mtrx = DataFrame()
        self.csh = {}
        self.ctrt = {}
        self.ctrt_ary = {}
        
    def iptctrt(self, prdt_name, tmpary):
        tmp_idx = [x[0] for x in tmpary]
        tmp_locno = [idx.sales.locno(x[0]) for x in tmpary]
        tmp_rate = [x[1] for x in tmpary]
        self.ctrt_ary[prdt_name] = tmpary
        self.ctrt[prdt_name] = DataFrame({'locno':tmp_locno,
                                          'rate':tmp_rate},
                                          index=tmp_idx)
        self.ctrt[prdt_name]['amt'] = self.ctrt[prdt_name]['rate'] \
                                      * self.mtrx.amt[prdt_name]
        self.ctrt[prdt_name]['odr'] = list(range(len(tmpary)))
        self.ctrt[prdt_name] = self.ctrt[prdt_name][['odr', 'locno', 'rate', 'amt']]
        
    def print_ctrt(self):
        print("Sales Contract Assumption")
        for key, val in self.ctrt.items():
            print("\n\n"+key)
            display(R(val))

    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
            
astns = Assumption_sales()

In [8]:
class Assumption_costs:
    def __init__(self):
        self.dct = {}
        self.amt = {}
        self.byname = {}
    
    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
    
astnc = Assumption_costs()

# Input Index Data

In [9]:
# Set period index
# prjt index : total project periods, and project base index
# cstrn index : construction index
# loan index : loan period index
# sales index : sales period index

tmp_col = ['idxname', 'start', 'periods']
tmp_ary = [['prjt',  '2021-04',   60],
           ['cstrn', '2022-03',   42],
           ['loan',  '2021-05',   55],
           ['sales', '2022-05',   40]]
astn.index = DataFrame(tmp_ary, columns=tmp_col)

idx = cf.PrjtIndex(idxname = list(astn.index.idxname),
                   start = list(astn.index.start),
                   periods = list(astn.index.periods + 1),
                   freq = 'M')

# Input Financing Condition Data

In [10]:
tmp_col =  ['title',  'amt_ntnl', 'amt_ntnl_once']
tmp_ary = [['equity',   10_000,        10_000]]
astn.equity = DataFrame(tmp_ary, columns=tmp_col)

equity = cf.Loan(idx, 
                 amt_ntnl = astn.equity.amt_ntnl[0],
                 amt_ntnl_once = astn.equity.amt_ntnl_once[0])

In [11]:
tmp_col = ['title', 'amt_ntnl', 'amt_ntnl_once', 'rate_fee', 'rate_IR']
tmp_ary = [['tra',    100_000,       10_000,        0.02,      0.043],
           ['trb',     30_000,       30_000,        0.06,      0.060],
           ['trc',      5_000,        5_000,        0.10,      0.100]]
astn.loan = DataFrame(tmp_ary, columns=tmp_col)

loan = cf.Intlz_loan(idx, idx.loan,
                     title = astn.loan.title,
                     amt_ntnl = astn.loan.amt_ntnl,
                     amt_ntnl_once = astn.loan.amt_ntnl_once,
                     rate_fee = astn.loan.rate_fee,
                     rate_IR = astn.loan.rate_IR)

# Input Sales Data
### 1) Sales Amount Matrix

In [12]:
# Sales Amount Matrix
tmp_col = ['prdtname', 'area_m2',  'prc_py',  'byname']
tmp_ary = [['lgstc',    244_377,     1.40,   '물류시설용지'],
           ['cmcl',      34_926,     2.90,   '상류시설용지'],
           ['hsg_sgl',    9_137,     2.80,   '단독주택'],
           ['hsg_apt',   33_134,     2.70,   '공동주택'],
           ['retail',    17_195,     3.90,   '상업시설용지'],
           ['office',     6_151,     3.90,   '업무시설용지'],
           ['parking',    8_352,     1.90,   '주차장용지'],
           ['gasstn',     2_400,     2.60,   '주유소용지'],
           ['rcclg',     10_000,     2.80,   '자원순환시설'],
           ['power',     10_000,     2.80,   '발전시설용지'],
           ['gsprfr',        25,     1.44,   '가스정압기'],
           ]

tmp_df = DataFrame(tmp_ary, columns=tmp_col)
tmp_df.set_index('prdtname', inplace=True)
tmp_df.loc[:,'area_py'] = tmp_df.area_m2 * PY
tmp_df.loc[:,'prc_m2'] = tmp_df.prc_py / PY
tmp_df.loc[:,'amt'] = tmp_df.area_py * tmp_df.prc_py
tmp_df = tmp_df[['byname', 'area_m2', 'area_py', \
                'prc_m2', 'prc_py', 'amt']]

astns.prdtname = [x[0] for x in tmp_ary]
astns.mtrx = tmp_df
R(astns.mtrx)

,byname,area_m2,area_py,prc_m2,prc_py,amt
prdtname,,,,,,
lgstc,물류시설용지,"244,377","73,924",4.6,1.4,"103,494"
cmcl,상류시설용지,"34,926","10,565",9.6,2.9,"30,639"
hsg_sgl,단독주택,"9,137","2,764",9.3,2.8,"7,739"
hsg_apt,공동주택,"33,134","10,023",8.9,2.7,"27,062"
retail,상업시설용지,"17,195","5,201",12.9,3.9,"20,286"
office,업무시설용지,"6,151","1,861",12.9,3.9,"7,257"
parking,주차장용지,"8,352","2,526",6.3,1.9,"4,800"
gasstn,주유소용지,"2,400",726,8.6,2.6,"1,888"
rcclg,자원순환시설,"10,000","3,025",9.3,2.8,"8,470"


### 2) Sales Contract Schedule

In [13]:
# Sales Contract Schedule
prdt_name = 'lgstc' # 물류시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.315],
           [idx.sales.locval(2023, 5), 0.274],
           [idx.sales.locval(2023, 11), 0.137],
           [idx.sales.locval(2024, 5), 0.137],
           [idx.sales.locval(2024, 11), 0.137]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'cmcl' # 상류시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.150],
           [idx.sales.locval(2023, 5), 0.200],
           [idx.sales.locval(2023, 11), 0.300],
           [idx.sales.locval(2024, 5), 0.300],
           [idx.sales.locval(2024, 11), 0.050]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'hsg_sgl' # 단독주택용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.300],
           [idx.sales.locval(2023, 5), 0.300],
           [idx.sales.locval(2023, 11), 0.200],
           [idx.sales.locval(2024, 5), 0.200],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'hsg_apt' # 공동주택용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 1.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'retail' # 상업시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.250],
           [idx.sales.locval(2023, 11), 0.300],
           [idx.sales.locval(2024, 5), 0.300],
           [idx.sales.locval(2024, 11), 0.150]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'office' # 업무시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.500],
           [idx.sales.locval(2023, 11), 0.500],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'parking' # 주차장용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 0.500],
           [idx.sales.locval(2024, 5), 0.500],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'gasstn' # 주유소용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 1.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'rcclg' # 자원순환시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 1.000],
           [idx.sales.locval(2023, 11), 0.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'power' # 발전시설용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 1.000],
           [idx.sales.locval(2023, 11), 0.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 0.000]]
astns.iptctrt(prdt_name, tmp_ary)

prdt_name = 'gsprfr' # 가스정압기용지
tmp_ary = [[idx.sales.locval(2022, 11), 0.000],
           [idx.sales.locval(2023, 5), 0.000],
           [idx.sales.locval(2023, 11), 0.000],
           [idx.sales.locval(2024, 5), 0.000],
           [idx.sales.locval(2024, 11), 1.000]]
astns.iptctrt(prdt_name, tmp_ary)

### 3) Sales Payment Schedule

In [14]:
# Payment Schedule(deposit, middle payment, balance)
astns.csh[0] = [[idx.sales.locval(2022, 11), 0.1],
                [idx.sales.locval(2023,  5), 0.2],
                [idx.sales.locval(2023, 11), 0.2],
                [idx.sales.locval(2024,  5), 0.2],
                [idx.sales.locval(2024, 11), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[1] = [[idx.sales.locval(2023,  5), 0.1],
                [idx.sales.locval(2023, 10), 0.2],
                [idx.sales.locval(2024,  3), 0.2],
                [idx.sales.locval(2024,  8), 0.2],
                [idx.sales.locval(2025,  1), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[2] = [[idx.sales.locval(2023, 11), 0.1],
                [idx.sales.locval(2024,  3), 0.2],
                [idx.sales.locval(2024,  7), 0.2],
                [idx.sales.locval(2024, 11), 0.2],
                [idx.sales.locval(2025,  3), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[3] = [[idx.sales.locval(2024,  5), 0.1],
                [idx.sales.locval(2024,  8), 0.2],
                [idx.sales.locval(2024, 11), 0.2],
                [idx.sales.locval(2025,  2), 0.2],
                [idx.sales.locval(2025,  5), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]
astns.csh[4] = [[idx.sales.locval(2024, 11), 0.1],
                [idx.sales.locval(2025,  1), 0.2],
                [idx.sales.locval(2025,  3), 0.2],
                [idx.sales.locval(2025,  5), 0.2],
                [idx.sales.locval(2025,  7), 0.2],
                [idx.sales.locval(2025,  9), 0.1]]

### 4) Initialize Sales Setting

In [15]:
sales = mng1.Intlz_sales_sellinlots(idx, idx.sales,
                                    slsmtrx=astns.mtrx,
                                    slsctrt=astns.ctrt,
                                    slscsh=astns.csh)

# Input Cost Data
### 1) Set Cost Class

In [16]:
cost = mng1.Intlz_costs(idx)

In [17]:
def inpt_cost_array(title, scdd_ary=None):
    cost_name = title[0]
    cost_byname = title[1]
    astnc.byname[cost_name] = cost_byname
    astnc.dct[cost_name] = None
    astnc.amt[cost_name] = 0
    
    if scdd_ary:
        astnc.dct[cost_name] = DataFrame(scdd_ary, columns=['scddidx', 'scddamt'])
        scddidx = astnc.dct[cost_name].scddidx
        scddamt = astnc.dct[cost_name].scddamt
        astnc.amt[cost_name] = astnc.dct[cost_name].scddamt.sum()
    else:
        scddidx = []
        scddamt = []
    
    cost.inptcst(cost_name, scddidx, scddamt)
    setattr(getattr(cost, cost_name), 'byname', cost_byname)

### 2) Land Purchase Costs

In [18]:
title = ('lnd_prchs', '용지매입비') # land purchase costs
tmp_ary = [[idx.locval(2021, 5), 23_712],
           [idx.locval(2021, 8),  2_587],
           [idx.locval(2022, 3),  6_575]]
inpt_cost_array(title, tmp_ary)

In [19]:
title = ('cmpstn_ostcs', '지장물보상비') # compensation for obstacles
tmp_ary = [[idx.locval(2021, 5), 177],
           [idx.locval(2021, 8), 19],
           [idx.locval(2022, 3), 49]]
inpt_cost_array(title, tmp_ary)

In [20]:
title = ('cmpstn_frmg', '영농보상비') # compensation for farming
tmp_ary = [[idx.locval(2021, 5), 1_191],
           [idx.locval(2021, 8), 130],
           [idx.locval(2022, 3), 330]]
inpt_cost_array(title, tmp_ary)

In [21]:
title = ('cmpstn_lnd', '토지보상용역비') # service fee for land compensation
tmp_ary = [[idx.locval(2021, 5), 361],
           [idx.locval(2021, 8), 39],
           [idx.locval(2022, 3), 100]]
inpt_cost_array(title, tmp_ary)

In [22]:
title = ('fee_aprsl', '감정평가수수료') # appraisal fee
tmp_ary = [[idx.locval(2021, 4), 23],
           [idx.locval(2021, 7), 132],
           [idx.locval(2021, 10), 66], 
           [idx.locval(2022, 3), 19.8],
           [idx.locval(2022, 6), 19.8],
           [idx.locval(2022, 9), 19.8],
           [idx.locval(2022, 12), 19.8],
           [idx.locval(2023, 3), 19.8],
           [idx.locval(2023, 6), 10]]
inpt_cost_array(title, tmp_ary)

In [23]:
title = ('rsv_cmpstn', '보상예비비') # reserve fund for compensation
tmp_ary = [[idx.locval(2021, 9), 345],
           [idx.locval(2021, 11), 345],
           [idx.locval(2022, 3), 1_151],
           [idx.locval(2022, 6), 460]]
inpt_cost_array(title, tmp_ary)

In [24]:
title = ('tax_aqstn', '소유권이전취득세') # acquisition tax
tmp_ary = [[idx.locval(2021, 5), 830],
           [idx.locval(2021, 8), 91],
           [idx.locval(2022, 3), 230]]
inpt_cost_array(title, tmp_ary)

In [25]:
title = ('fee_jdcl', '법무사수수료') # fee for a judicial scrivener
tmp_ary = [[idx.locval(2021, 5), 95],
           [idx.locval(2021, 8), 10],
           [idx.locval(2022, 3), 26]]
inpt_cost_array(title, tmp_ary)

In [26]:
title = ('tax_lndctgry', '지목변경취득세') # acquisition tax for change of land category
tmp_ary = [[idx.locval(2025, 8), 3_015]]
inpt_cost_array(title, tmp_ary)

In [27]:
title = ('tax_cprhsvlnd', '종합부동산세') # comprehensive real estate tax
tmp_ary = [[idx.locval(2021, 12), 63],
           [idx.locval(2022, 12), 118],
           [idx.locval(2023, 12), 126],
           [idx.locval(2024, 12), 138],
           [idx.locval(2025, 12), 149],]
inpt_cost_array(title, tmp_ary)

In [28]:
title = ('tax_prpty', '재산세') # Property tax
tmp_ary = [[idx.locval(2021, 9), 85],
           [idx.locval(2022, 9), 124],
           [idx.locval(2023, 9), 130],
           [idx.locval(2024, 9), 137],
           [idx.locval(2025, 6), 144],]
inpt_cost_array(title, tmp_ary)

In [29]:
title = ('ctbn_lnd', '용지부담금') # contribution cost of land
tmp_ary = [[idx.locval(2021,  5), 1_067],
           [idx.locval(2021,  8), 1_008],
           [idx.locval(2021, 11), 1_344]]
inpt_cost_array(title, tmp_ary)

### 3) Construction Costs

In [30]:
title = ('cstrn_dvlp', '조성공사비') # construction cost for development
tmp_ary = [[idx.locval(2021,  5), 9_606],
           [idx.locval(2021,  8),   108],
           [idx.locval(2021, 11),   180],
           [idx.locval(2022,  1),    72],
           [idx.locval(2022,  3), 7_029],
           [idx.locval(2022,  5), 2_017],
           [idx.locval(2022,  7),   674],
           [idx.locval(2022,  9), 1_012],
           [idx.locval(2022, 11), 1_054],
           [idx.locval(2023,  1), 2_017],
           [idx.locval(2023,  3), 2_626],
           [idx.locval(2023,  5), 3_953],
           [idx.locval(2023,  7), 3_915],
           [idx.locval(2023,  9), 4_559],
           [idx.locval(2023, 11), 5_596],
           [idx.locval(2024,  1), 5_859],
           [idx.locval(2024,  3), 6_438],
           [idx.locval(2024,  5), 6_355],
           [idx.locval(2024,  7), 5_137],
           [idx.locval(2024,  9), 4_477],
           [idx.locval(2024, 11), 3_239],
           [idx.locval(2024, 12),   110],
           [idx.locval(2025,  1), 2_049],
           [idx.locval(2025,  3), 2_467],
           [idx.locval(2025,  5), 5_151],
           [idx.locval(2025,  6),   275],
           [idx.locval(2025,  7),    21],
           [idx.locval(2025,  8), 1_100],
           [idx.locval(2025,  9),    21],
           [idx.locval(2025, 11),    48],
           [idx.locval(2025, 12),   235],
           ]
inpt_cost_array(title, tmp_ary)

### 4) Sales Fee Costs

In [31]:
title = ('sales_fee', '분양대행수수료') # sales fee
inpt_cost_array(title)
cost.sales_fee.rate = 0.04

# Execution Cash Flow

In [32]:
acc = mng1.Intlz_accounts(idx, ['oprtg', 'sales', 'repay'])

In [33]:
# Execute cash flow
for idxno in idx.index:
    #### Costs : Set cost management class ####
    mng_cst = mng1.Mngmnt_cst(idxno, cost)
    
    #### Loans : Set loan withdrawble ####
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno, idx[0])
    loan.tra.set_wtdrbl_intldate(idxno)
    loan.trb.set_wtdrbl_intldate(idxno)
    loan.trc.set_wtdrbl_intldate(idxno)
    
    #### Sales : Get money from sales products ####
    # 분양수입대금 sales schedule에 따라 운영계좌로 이체
    for prdt in sales.prdtlst:
        sls_prdt = getattr(sales, prdt)
        mng_prdt = mng1.Mngmnt_sls(idxno, sls_prdt)
        mng_prdt.make_ctrt_plan()
        sls_amt_ctrt = sls_prdt.amt_add[idxno]
        sls_amt_csh = sls_prdt.sub_rsdl_cum[idxno]
        
        # Send cash on sales to sales account
        sls_prdt.send(idxno, sls_amt_csh, acc.sales)
        
        # Calculate sales fee amount
        mng_cst.cal_slsfee(sls_amt_ctrt)
        
    #### Sales : Allocate sales amount ####
    # 분양수입금 분배
    intl_bal_end = acc.sales.bal_end[idxno]
    if not loan.ttl.is_repaid:
        amt_to_oprtg = intl_bal_end * 0.2
        amt_to_repay = intl_bal_end * 0.8
        acc.sales.send(idxno, amt_to_oprtg, acc.oprtg)
        acc.sales.send(idxno, amt_to_repay, acc.repay)
    elif loan.ttl.is_repaid:
        acc.sales.send(idxno, intl_bal_end, acc.oprtg)
        amt_trsfr = cf.limited(acc.repay.bal_end[idxno], lower=[0])
        acc.repay.send(idxno, amt_trsfr, acc.oprtg)
        
    #### Costs : Calculate expected operating cost ####
    cst_oprtg = 0
    cst_oprtg = mng_cst.cst_oprtg
    
    #### Loans : Calculate expected financial cost ####
    mng_fncl = {}
    cst_fncl = 0
    for trch, loan_each in loan.dct.items():
        mng_fncl[trch] = mng1.Mngmnt_fncl(idxno, loan_each)
        cst_fncl += mng_fncl[trch].amt_scdd
    
    #### Loans : Withdraw loan ####    
    cst_all = cst_oprtg + cst_fncl # 총 지출 필요금액 계산
    
    wtdrw = mng1.Mngmnt_wtdrw(idxno, acc.oprtg)
    wtdrw.cal_amt_exptd(cst_all) # 대출금 인출 필요금액 계산
    wtdrw.wtdrw_eqty(equity, equity.amt_ntnl_once)
    if idxno == idx.loan[0]:
        wtdrw.wtdrw_loan(loan.trc, loan.trc.amt_ntnl_once)
        wtdrw.wtdrw_loan(loan.trb, loan.trb.amt_ntnl_once)
        wtdrw.wtdrw_loan(loan.tra, loan.tra.amt_ntnl_once)
    wtdrw.wtdrw_loan(loan.trc)
    wtdrw.wtdrw_loan(loan.trb)
    wtdrw.wtdrw_loan(loan.tra)
    
    #### Costs : 토지비, 공사비 등 각종 비용 지출 ####
    mng_cst.pay_oprtgcst(acc.oprtg)
    
    #### Loans : Pay financial cost ####
    mng_fncl['tra'].pay_all(acc.oprtg)
    mng_fncl['trb'].pay_all(acc.oprtg)
    mng_fncl['trc'].pay_all(acc.oprtg)
    
    #### Loans : Repay loan amounts ####
    rpymngA = mng1.Mngmnt_repay(idxno, loan.tra, acc.oprtg, acc.repay)
    rpymngA.repayment_process()
    
    if loan.tra.is_repaid:
        rpymngB = mng1.Mngmnt_repay(idxno, loan.trb, acc.oprtg, acc.repay)
        rpymngB.repayment_process()
    if loan.trb.is_repaid:
        rpymngC = mng1.Mngmnt_repay(idxno, loan.trc, acc.oprtg, acc.repay)
        rpymngC.repayment_process()
        
    #### Loans : Set back loan unwithdrawble at maturity ####
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    loan.tra.setback_wtdrbl_mtrt(idxno)
    loan.trb.setback_wtdrbl_mtrt(idxno)
    loan.trc.setback_wtdrbl_mtrt(idxno)

In [38]:
R(acc.oprtg.df)

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-04-30,0.0,0.0,0,"10,000",23,"9,977"
2021-05-31,0.0,0.0,"9,977","45,000","41,339","13,638"
2021-06-30,0.0,0.0,"13,638",0,227,"13,410"
2021-07-31,0.0,0.0,"13,410",0,360,"13,051"
2021-08-31,0.0,0.0,"13,051",0,"4,220","8,832"
2021-09-30,0.0,0.0,"8,832",0,657,"8,174"
2021-10-31,0.0,0.0,"8,174",0,294,"7,880"
2021-11-30,0.0,0.0,"7,880",0,"2,096","5,784"
2021-12-31,0.0,0.0,"5,784",0,290,"5,494"
2022-01-31,0.0,0.0,"5,494",0,300,"5,194"
